In [2]:
## Libaries ##
##############
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show
import os, glob
import pandas as pd
import numpy as np
from IPython.display import display
import ipywidgets as widgets
import nibabel as nib
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.models import load_model, Model
from model.metrics import dice_coef, dice_coef_loss
from model.model_code import UNET

### Interactive Image & Mask Viewer

In [45]:
### Widget Slider ###
#####################
imgPath = 'data/volumes/training_volumes/img/'
lblPath = 'data/volumes/training_volumes/mask/'
img_fnames = os.listdir(imgPath)
lbl_fnames = os.listdir(lblPath)
img_data = nib.load(os.path.join(imgPath, img_fnames[0])).get_fdata()
mask_data = nib.load(os.path.join(lblPath, lbl_fnames[0])).get_fdata()
mask_data[mask_data > 1] = 1 # 2.0 values are converted to 1.0 so that its just binary mask
img_data = normalizeImageIntensityRange(img_data)# 0-1 Normalize intensity range of image
print (f'Image:    shape={img_data.shape}    min-max= {np.min(img_data)}->{np.max(img_data)}        {type(img_data)}')
print (f' Mask:    shape={mask_data.shape}    min-max= {np.min(mask_data)}->{np.max(mask_data)}        {type(mask_data)}')
# Define a function to update the plot when the z slider is changed
def update_plot(z, show_mask):
    plt.figure(figsize=(5, 5))
    plt.imshow(img_data[:, :, z], cmap='gray')
    if show_mask:
        plt.imshow(mask_data[:, :, z], cmap='Greens', alpha=0.4)  # Overlay mask with some transparency
    plt.title(f'coronal (z)-{z}')
    plt.show()
# Create slider for z dimension
z_slider = widgets.IntSlider(min=0, max=img_data.shape[2]-1, step=1, value=img_data.shape[2]//2, description='Z')
# Create toggle button for mask
mask_toggle = widgets.ToggleButton(
    value=True,
    description='Show Mask',
    disabled=False,
    button_style='', 
    tooltip='Toggle Mask Display',
    icon='eye'
)
# Display the slider, toggle button, and the plot
widgets.interactive(update_plot, z=z_slider, show_mask=mask_toggle)




Image:    shape=(256, 256, 16)    min-max= 0.0->1.0        <class 'numpy.ndarray'>
 Mask:    shape=(256, 256, 16)    min-max= 0.0->1.0        <class 'numpy.ndarray'>


interactive(children=(IntSlider(value=8, description='Z', max=15), ToggleButton(value=True, description='Show …

### Save 3D Volumes Into 2D Slices 

In [7]:
# FUNCTIONS & setting parameters # ################################################################################################# 
import numpy.ma as ma

# Constants
SLICE_X = False
SLICE_Y = False
SLICE_Z = True

# 0-1 Normalize Image Intensity Range
def normalizeImageIntensityRange(img): # 0-1 Normalize intensity range of image
    # img = img / np.max(img) 
    img_min = np.min(img)
    img_max = np.max(img)
    img_range = img_max - img_min
    return (img - img_min) / img_range

# Read image or mask volume
def loadVolume(volumePath, binarize_mask=False, normalize=False):
    volume = nib.load(volumePath).get_fdata()
    if binarize_mask == True: #for Mask/Labels
        volume[volume > 1] = 1 #values of 2.0 are converted to 1.0 so that just binary mask
    if normalize == True:
        volume = normalizeImageIntensityRange(volume)
    return volume

# Save volume slice to file
def saveSlice(vol_slice, fname, path):
    image = np.uint8(vol_slice * 255)
    fout = os.path.join(path, f'{fname}.png')
    if os.path.exists(fout):
        print(f"Overwriting file: [+] Slice saved: {fout}", end="\r")
    else:
        print(f"[+] Slice saved: {fout}", end="\r")  
    cv2.imwrite(fout, image)

# Slice image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    #print(dimx, dimy, dimz)
    cnt = 0
    if SLICE_X == True:
        cnt += dimx
        #print('Slicing X: ')
        for d in range(dimx):
            saveSlice(vol_slice=vol[d,:,:], fname=fname[:20]+f"_slice{str(d).zfill(3)}-x"+fname[20:], path=path)
    if SLICE_Y == True:
        cnt += dimy
        #print('Slicing Y: ')
        for d in range(dimy):
            saveSlice(vol_slice=vol[:,d,:], frame=fname[:20]+f"_slice{str(d).zfill(3)}-y"+fname[20:], path=path)
    if SLICE_Z == True:
        cnt += dimz
        #print('Slicing Z: ')
        for d in range(dimz):
            saveSlice(vol_slice=vol[:,:,d], fname=fname[:20]+f"_slice{str(d).zfill(3)}-z"+fname[20:], path=path)
    return cnt
# ################################################################################################# Functions ###

In [8]:
### TRAINING DATA: Read and process image and mask volumes ###
##############################################################
path = 'data/volumes/training_volumes'
image_files = sorted(glob.glob(f'{path}/img/*avw.nii.gz')) # Get a list of all image files
label_files = sorted(glob.glob(f'{path}/mask/*label.nii.gz')) # Get a list of all label files
assert len(image_files) == len(label_files), "Mismatch in number of image and label files" # Ensure that the number of image files and label files are the same

count=0
for subnum in range(len(image_files)):
    # load MRI
    imgpath = image_files[subnum]
    lblpath = label_files[subnum]
    img = loadVolume(imgpath, normalize=True)
    lbl = loadVolume(lblpath, binarize_mask=True)
    print (f'Image[{str(subnum).zfill(2)}] - {type(img)} {img.shape}    min/max = {np.min(img)} < {np.max(img)}    {imgpath}')
    print (f'Label[{str(subnum).zfill(2)}] - {type(lbl)} {lbl.shape}    min/max = {np.min(lbl)} < {np.max(lbl)}    {lblpath}')
    
    # Slicing and saving
    if img.shape[-1] >= 14 and img.shape == lbl.shape:#excluding scans with dim3 < 14 and make sure img&lbl is same shape
        #images.append(img[:,:,:14])#example of also Cropping: slice dim3 <= 14 
        Volume_image_fname = 'trnVolume'+str(subnum).zfill(3)+'__IMAGE__'+((imgpath.split('\\'))[1])[:-7]
        Volume_label_fname = 'trnVolume'+str(subnum).zfill(3)+'__LABEL__'+((lblpath.split('\\'))[1])[:-7]
        Icnt = sliceAndSaveVolumeImage(vol=img, fname=Volume_image_fname, path='data/slices/training_slices/img/')# Icnt = sliceAndSaveVolumeImage(vol=img, fname=f"image{str(subnum).zfill(2)}_({iname})", path='data/slices/training_slices/img/')
        Lcnt = sliceAndSaveVolumeImage(vol=lbl, fname=Volume_label_fname, path='data/slices/training_slices/mask/')# Lcnt = sliceAndSaveVolumeImage(vol=lbl, fname=f"label{str(subnum).zfill(2)}_({lname})", path='data/slices/training_slices/mask/')
        # print(f"\n{'Volume'+str(subnum).zfill(3)+'__'}... Lcnt({Lcnt}) & Icnt({Icnt}) slices created for images and labels respectivley\n")
        count += Icnt
    else:
        print (f'Image[{str(subnum).zfill(2)}] - SKIPPED SLICING & SAVING VOLUMES')
print("\n\n\nAmount of Training Slices (2D images) =", count)
    
    

Image[00] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/img\7_BL6_m_RC_tk07_3wk_Kl1_Exp006_Pro001avw.nii.gz
Label[00] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/mask\7_BL6_m_RC_tk07_3wk_Kl1_Exp006_Pro001avw-label.nii.gz
Image[01] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/img\7_BL6_m_RC_tk16_3wk_Kn1_Exp006_Pro001avw.nii.gz
Label[01] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/mask\7_BL6_m_RC_tk16_3wk_Kn1_Exp006_Pro001avw-label.nii.gz
Image[02] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/img\7_BL6_m_RC_tk17_3wk_Kn1_Exp005_Pro001avw.nii.gz
Label[02] - <class 'numpy.ndarray'> (256, 256, 14)    min/max = 0.0 < 1.0    data/volumes/training_volumes/mask\7_BL6_m_RC_tk17_3wk_Kn1_Exp005_Pro001avw-label.nii.gz
Image[03] - <class 'numpy

In [9]:
### TEST DATA: Read and process image and mask volumes ###
##########################################################
path = 'data/volumes/test_volumes'
image_files = sorted(glob.glob(f'{path}/img/*avw.nii.gz')) # Get a list of all image files
label_files = sorted(glob.glob(f'{path}/mask/*.nii.gz')) # Get a list of all label files
assert len(image_files) == len(label_files), "Mismatch in number of image and label files" # Ensure that the number of image files and label files are the same
count=0
for subnum in range(len(image_files)):
    # load MRI
    imgpath = image_files[subnum]
    lblpath = label_files[subnum]
    img = loadVolume(imgpath, normalize=True)
    lbl = loadVolume(lblpath, binarize_mask=True)
    print (f'Image[{str(subnum).zfill(2)}] - {type(img)} {img.shape}    min/max = {np.min(img)} < {np.max(img)}    {imgpath}')
    print (f'Label[{str(subnum).zfill(2)}] - {type(lbl)} {lbl.shape}    min/max = {np.min(lbl)} < {np.max(lbl)}    {lblpath}')
    
    # Slicing and saving
    if img.shape[-1] >= 14 and img.shape == lbl.shape:#excluding scans with dim3 < 14 and make sure img&lbl is same shape
        Volume_image_fname = 'tstVolume'+str(subnum).zfill(3)+'__IMAGE__'+((imgpath.split('\\'))[1])[:-7]
        Volume_label_fname = 'tstVolume'+str(subnum).zfill(3)+'__LABEL__'+((lblpath.split('\\'))[1])[:-7]
        Icnt = sliceAndSaveVolumeImage(vol=img, fname=Volume_image_fname, path='data/slices/test_slices/img/')# Icnt = sliceAndSaveVolumeImage(vol=img, fname=f"image{str(subnum).zfill(2)}_({iname})", path='data/slices/training_slices/img/')
        Lcnt = sliceAndSaveVolumeImage(vol=lbl, fname=Volume_label_fname, path='data/slices/test_slices/mask/')# Lcnt = sliceAndSaveVolumeImage(vol=lbl, fname=f"label{str(subnum).zfill(2)}_({lname})", path='data/slices/training_slices/mask/')
        # print(f"\n{'Volume'+str(subnum).zfill(3)+'__'}... Lcnt({Lcnt}) & Icnt({Icnt}) slices created for images and labels respectivley\n")
        count += Icnt
    else:
        print (f'Image[{str(subnum).zfill(2)}] - SKIPPED SLICING & SAVING VOLUMES')
print("\n\n\nAmount of Test Slices (2D images) =", count)
    
    

Image[00] - <class 'numpy.ndarray'> (256, 256, 20)    min/max = 0.0 < 1.0    data/volumes/test_volumes/img\12WD1A_ETKV_1_1_Su1_Exp003_Pro001avw.nii.gz
Label[00] - <class 'numpy.ndarray'> (256, 256, 20)    min/max = 0.0 < 1.0    data/volumes/test_volumes/mask\12WD1A_ETKV_1_1_Su1_Exp003_Pro001_RX_SEG.nii.gz
Image[01] - <class 'numpy.ndarray'> (256, 256, 20)    min/max = 0.0 < 1.0    data/volumes/test_volumes/img\12WD1A_ETKV_2_1_Su1_Exp004_Pro001avw.nii.gz
Label[01] - <class 'numpy.ndarray'> (256, 256, 20)    min/max = 0.0 < 1.0    data/volumes/test_volumes/mask\12WD1A_ETKV_2_1_Su1_Exp004_Pro001_RX_SEG.nii.gz
Image[02] - <class 'numpy.ndarray'> (256, 256, 18)    min/max = 0.0 < 1.0    data/volumes/test_volumes/img\12WD1A_ETKV_6_1_Su1_Exp004_Pro001avw.nii.gz
Label[02] - <class 'numpy.ndarray'> (256, 256, 18)    min/max = 0.0 < 1.0    data/volumes/test_volumes/mask\12WD1A_ETKV_6_1_Su1_Exp004_Pro001_RX_SEG.nii.gz
Image[03] - <class 'numpy.ndarray'> (256, 256, 18)    min/max = 0.0 < 1.0    da